## 1. Import Library

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn

import albumentations as albu
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset

import sklearn.model_selection as sk

import warnings
warnings.filterwarnings('ignore')

## 2. Import Data

In [2]:
########## 코랩 환경 ##########

# from google.colab import drive
# drive.mount('/content/drive')

# # Dataset path load
# train_path = '/content/drive/MyDrive/Test_sample'

In [3]:
########## 주피터 환경 ##########

test_path = 'TestData_sample/Test_sample/'

## 3. Data Preprocessing

## 3.1 Normalization

In [4]:
data=[]
label=[]

files = os.listdir(test_path)
files.sort()

for f in files:
    img = Image.open(test_path+f)
    one_img = np.asarray(np.float32(img))
    norm_img = one_img/255.0

    data.append([norm_img])  
    label.append(0)

        
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

test_X = torch.from_numpy(data)
test_Y = torch.from_numpy(label)

test_X = test_X.type(torch.cuda.FloatTensor)
test_Y = test_Y.type(torch.cuda.LongTensor)

test_dataset = TensorDataset(test_X,test_Y)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
print(test_X.shape, test_Y.shape)

torch.Size([30, 1, 100, 100]) torch.Size([30])


## 4. Model Import & Train

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            
            nn.Conv2d(1, 16, 3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            nn.Conv2d(16, 16, 3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            nn.MaxPool2d(3,2),
            
            nn.Conv2d(16, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.MaxPool2d(3,2),
            
            nn.Conv2d(32, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.MaxPool2d(3,2),

        )
        
        self.fc = nn.Sequential(
            
            nn.Linear(4096, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(512, 3)

        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x


model = Net()
model.cuda()

Net(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (15): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [7]:
########## 코랩 환경 ##########

# model = torch.load('/content/drive/MyDrive/Model_TeamC')

In [8]:
########## 주피터 환경 ##########

model = torch.load('Model_TeamC')

In [9]:
# evaluation
model.eval()
f_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = Variable(inputs), Variable(labels)
        output = model(inputs)

        idx, pred = output.max(1)
        pred = (pred.detach().cpu().numpy())

        f_pred.extend(pred.tolist())

# 5. Submission

In [10]:
# ########## 코랩 환경 ##########

# # Submission file
# img_files = [i+1 for i in range(len(f_pred))]
# submission = pd.DataFrame(index = range(0,len(f_pred)), columns = {'img','pred_class'})
# submission['img'] = img_files
# submission['pred_class'] = f_pred
# # pd.to_csv('/content/drive/MyDrive/Submission_TeamC.csv')
# submission

In [12]:
########## 주피터 환경 ##########

# Submission file
img_files = [i+1 for i in range(len(f_pred))]
submission = pd.DataFrame(index = range(0,len(f_pred)), columns = {'img','pred_class'})
submission['img'] = img_files
submission['pred_class'] = f_pred
# pd.to_csv('Submission_TeamC.csv')
submission

,pred_class,img
0,2,1
1,0,2
2,2,3
3,0,4
4,1,5
5,0,6
6,1,7
7,0,8
8,1,9
9,2,10
